<a href="https://colab.research.google.com/github/LeeGitaek/Kaggle_Solving/blob/master/seoul_safety.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#서울시 생활안전도에 따른 생활환경 만족도 예측

In [1]:
!pip uninstall --y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6

Uninstalling kaggle-1.5.6:
  Successfully uninstalled kaggle-1.5.6
     |████████████████████████████████| 1.5MB 2.8MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 58 kB 1.7 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-py3-none-any.whl size=72859 sha256=5c5d9a9499405da2cd75da809487cb222b28c5fc143b685a90b83604fd041ff7
  Stored in directory: /root/.cache/pip/wheels/01/3e/ff/77407ebac3ef71a79b9166a8382aecf88415a0bcbe3c095a01
Successfully built kaggle


In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle
!ls lha kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

ls: cannot access 'lha': No such file or directory
kaggle.json


In [4]:
!kaggle competitions download -c ai-project-life-environment

  0% 0.00/5.42k [00:00<?, ?B/s]
100% 5.42k/5.42k [00:00<00:00, 4.08MB/s]


In [5]:
!unzip ai-project-life-environment

Archive:  ai-project-life-environment.zip
  inflating: submit sample.csv       
  inflating: test data.csv           
  inflating: train data.csv          


In [383]:
import pandas as pd
import numpy as np
import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.nn as nn

import random

from sklearn import preprocessing

In [384]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [385]:
learning_rate = 0.5
training_epochs = 2000
batch_size = 100

In [386]:
train = pd.read_csv('train data.csv')
print(train.head(10))
print(train.info())

     Year  Noise  Air pollution  ...  Garbage  Parking order  Satisfaction
0  201101   27.7           27.9  ...     35.1           48.8          61.0
1  201102   26.9           28.1  ...     35.0           50.1          61.6
2  201103   30.3           26.9  ...     35.3           44.1          58.8
3  201104   29.6           27.0  ...     34.2           47.0          55.0
4  201105   28.2           27.1  ...     38.0           52.0          60.3
5  201106   28.6           30.5  ...     36.7           51.1          60.9
6  201107   27.5           29.0  ...     33.8           49.2          62.5
7  201108   25.5           24.6  ...     31.8           42.9          62.3
8  201109   28.6           24.8  ...     34.9           46.2          60.4
9  201110   28.5           27.5  ...     37.6           50.3          60.3

[10 rows x 9 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 275 entries, 0 to 274
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
--

In [387]:
test = pd.read_csv('test data.csv')
print(test.head(10))
print(test.info())

     Year  Noise  Air pollution  ...  Crime  Garbage  Parking order
0  201701   34.8           36.6  ...   44.8     48.0           51.6
1  201702   34.9           35.8  ...   28.1     37.8           46.9
2  201703   38.1           42.0  ...   37.1     42.4           48.5
3  201704   33.1           43.1  ...   33.5     37.2           48.8
4  201705   29.8           34.1  ...   33.3     45.2           51.4
5  201706   29.6           33.2  ...   28.9     36.9           39.1
6  201707   28.3           38.8  ...   33.8     47.2           51.0
7  201708   34.9           44.5  ...   45.5     44.7           49.4
8  201709   30.8           31.8  ...   39.5     34.9           48.9
9  201710   25.1           29.0  ...   25.5     34.3           41.0

[10 rows x 8 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Year             25 non-null  

In [388]:
x_train = train.iloc[:,1:-1]
y_train = train.iloc[:,[-1]]

x_train = np.array(x_train)
y_train = np.array(y_train)

Scaler = preprocessing.StandardScaler()
x_train = Scaler.fit_transform(x_train)

x_train = torch.FloatTensor(x_train)
y_train = torch.FloatTensor(y_train)

print(x_train.shape)
print(y_train.shape)


torch.Size([275, 7])
torch.Size([275, 1])


In [389]:
train_dataset = torch.utils.data.TensorDataset(x_train,y_train)


data_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle = True,
                                          drop_last = True)


In [390]:
class MishFunction(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x):
        ctx.save_for_backward(x)
        return x * torch.tanh(F.softplus(x))   # x * tanh(ln(1 + exp(x)))

    @staticmethod
    def backward(ctx, grad_output):
        x = ctx.saved_variables[0]
        sigmoid = torch.sigmoid(x)
        tanh_sp = torch.tanh(F.softplus(x)) 
        return grad_output * (tanh_sp + x * sigmoid * (1 - tanh_sp * tanh_sp))

class Mish(nn.Module):
    def forward(self, x):
        return MishFunction.apply(x)

def to_Mish(model):
    for child_name, child in model.named_children():
        if isinstance(child, nn.ReLU):
            setattr(model, child_name, Mish())
        else:
            to_Mish(child)

In [391]:
linear1 = torch.nn.Linear(7,16, bias = True)
linear2 = torch.nn.Linear(16,16, bias = True)
linear3 = torch.nn.Linear(16,16, bias = True)
linear4 = torch.nn.Linear(16,16, bias = True)
linear5 = torch.nn.Linear(16,1, bias = True)
mish = Mish()
#dropout = torch.nn.Dropout(p=0.3)

In [392]:
torch.nn.init.kaiming_uniform_(linear1.weight)
torch.nn.init.kaiming_uniform_(linear2.weight)
torch.nn.init.kaiming_uniform_(linear3.weight)
torch.nn.init.kaiming_uniform_(linear4.weight)
torch.nn.init.kaiming_uniform_(linear5.weight)

Parameter containing:
tensor([[-0.5368,  0.3833, -0.1810,  0.2788, -0.0437, -0.5479,  0.3200, -0.0196,
         -0.3081,  0.0012,  0.2445,  0.4331, -0.3220,  0.3320, -0.0735, -0.4077]],
       requires_grad=True)

In [393]:
model = torch.nn.Sequential(linear1, mish, 
                            linear2, mish, 
                            linear3, mish,
                            linear4, mish,
                            linear5).to(device)

In [394]:
loss = torch.nn.MSELoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 

In [395]:
total_batch = len(data_loader)
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:

        X = X.to(device)
        Y = Y.to(device)

        # 그래디언트 초기화
        optimizer.zero_grad()
        # Forward 계산
        hypothesis = model(X)
        # Error 계산
        cost = loss(hypothesis, Y)
        # Backparopagation
        cost.backward()
        # 가중치 갱신
        optimizer.step()

        # 평균 Error 계산
        avg_cost += cost / total_batch

    print('Epoch:', '%03d' % (epoch + 1), 'rate =', '{:.4f}'.format(avg_cost))

print('Learning finished')

Epoch: 001 rate = 6422.2388
Epoch: 002 rate = 4103.4092
Epoch: 003 rate = 3327.5312
Epoch: 004 rate = 7478.0889
Epoch: 005 rate = 7646.8633
Epoch: 006 rate = 1917.7495
Epoch: 007 rate = 439.1527
Epoch: 008 rate = 586.2947
Epoch: 009 rate = 395.3756
Epoch: 010 rate = 342.0200
Epoch: 011 rate = 220.0643
Epoch: 012 rate = 266.8322
Epoch: 013 rate = 225.5745
Epoch: 014 rate = 181.8834
Epoch: 015 rate = 164.8474


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: 'saved_variables' is deprecated; use 'saved_tensors'
  if __name__ == '__main__':


Epoch: 016 rate = 103.0188
Epoch: 017 rate = 120.4312
Epoch: 018 rate = 64.4683
Epoch: 019 rate = 103.0639
Epoch: 020 rate = 72.8508
Epoch: 021 rate = 72.8528
Epoch: 022 rate = 65.2356
Epoch: 023 rate = 43.5909
Epoch: 024 rate = 50.7714
Epoch: 025 rate = 42.3675
Epoch: 026 rate = 38.9301
Epoch: 027 rate = 54.4779
Epoch: 028 rate = 20.3423
Epoch: 029 rate = 27.9244
Epoch: 030 rate = 21.9495
Epoch: 031 rate = 12.6147
Epoch: 032 rate = 19.3098
Epoch: 033 rate = 15.6769
Epoch: 034 rate = 16.0874
Epoch: 035 rate = 13.6073
Epoch: 036 rate = 15.0496
Epoch: 037 rate = 10.5854
Epoch: 038 rate = 9.0452
Epoch: 039 rate = 11.7846
Epoch: 040 rate = 7.8974
Epoch: 041 rate = 8.1571
Epoch: 042 rate = 7.6846
Epoch: 043 rate = 7.2797
Epoch: 044 rate = 7.3487
Epoch: 045 rate = 6.5086
Epoch: 046 rate = 6.2280
Epoch: 047 rate = 5.8419
Epoch: 048 rate = 6.8276
Epoch: 049 rate = 6.7596
Epoch: 050 rate = 7.3808
Epoch: 051 rate = 6.7934
Epoch: 052 rate = 6.2342
Epoch: 053 rate = 6.6371
Epoch: 054 rate = 6.3220

In [396]:
test_xy = pd.read_csv("test data.csv",header=None, skiprows=1)

with torch.no_grad():
  x_test_data=test_xy.iloc[:,1:]
  x_test_data=np.array(x_test_data)
  x_test_data = Scaler.transform(x_test_data)
  x_test_data=torch.from_numpy(x_test_data).float().to(device)

  prediction = model(x_test_data)

In [397]:
correct_prediction = prediction.cpu().numpy().reshape(-1,1)

In [398]:
submit=pd.read_csv('submit sample.csv')

In [399]:
for i in range(len(correct_prediction)):
  submit['Expected'][i]=correct_prediction[i].item()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [400]:
submit.to_csv('submit.csv',index=False,header=True)

In [401]:
!kaggle competitions submit -c ai-project-life-environment -f submit.csv -m "14010974_이기택"

100% 549/549 [00:07<00:00, 72.3B/s]
Successfully submitted to 인공지능 텀프로젝트